## GENERATING A SONG

In [1]:
import os
import music21
from music21 import converter, note, chord
import pickle
import numpy as np
from keras.models import load_model

In [2]:
model = load_model('model/model_checkpoint.keras')

In [3]:
# Define constants
SEQUENCE_LENGTH = 100  # Adjust as needed
EPOCHS = 100  # Adjust as needed
BATCH_SIZE = 128  # Adjust as needed
OUTPUT_FILE = 'output.mid'

# Load Chopin MIDI files
data_folder = 'data'
files = [os.path.join(data_folder, file) for file 
         in os.listdir(data_folder) if file.endswith('.midi') or file.endswith('.mid')]

def get_notes():
    """ Get all the notes and chords from the MIDI files in the specified directory """
    notes = []

    for file in files:
        midi = converter.parse(file)
        print("Parsing %s" % file)
        notes_to_parse = midi.flatten()
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.pitches))
    return notes

notes_file_path = 'data/notes'
notes = []

if os.path.exists(notes_file_path):
    # Loading notes from the file
    with open('data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)
else:
    # createing a list of ntoes
    notes = get_notes()
    # saving notes to the file:
    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

# Create input sequences and corresponding output
unique_notes = sorted(set(notes))
note_to_int = dict((note, number) for number, note in enumerate(unique_notes))
int_to_note = dict((number, note) for number, note in enumerate(unique_notes))

input_sequences = []
output_sequences = []
for i in range(len(notes) - SEQUENCE_LENGTH):
    sequence_in = notes[i:i + SEQUENCE_LENGTH]
    sequence_out = notes[i + SEQUENCE_LENGTH]
    input_sequences.append([note_to_int[char] for char in sequence_in])
    output_sequences.append(note_to_int[sequence_out])


In [4]:
start = np.random.randint(0, len(input_sequences) - 1)

In [5]:
pattern = input_sequences[start]
prediction_output = []
number_of_notes_to_generate = 100
for i in range(number_of_notes_to_generate):
    input_sequence = np.reshape(pattern, (1, len(pattern), 1))
    input_sequence = input_sequence / float(len(unique_notes))

    prediction = model.predict(input_sequence, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    print(f"{i}/{number_of_notes_to_generate} | {result}")
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:]

0/100 | C#5.B4
1/100 | B3
2/100 | D3
3/100 | G#4
4/100 | F5
5/100 | F4
6/100 | F#2
7/100 | A4
8/100 | B-3
9/100 | G#4
10/100 | G#4
11/100 | F5
12/100 | F3
13/100 | B4
14/100 | A2
15/100 | B-4
16/100 | F5
17/100 | B3
18/100 | C#3
19/100 | C#4
20/100 | B4
21/100 | F5
22/100 | D5
23/100 | F3
24/100 | F3
25/100 | C#5
26/100 | C#4
27/100 | C#4
28/100 | F3
29/100 | F5
30/100 | F5
31/100 | F#4
32/100 | B3
33/100 | C#4
34/100 | B-4
35/100 | F3
36/100 | F3
37/100 | D4
38/100 | F3
39/100 | C#5
40/100 | D4
41/100 | B-4
42/100 | F3
43/100 | B-3
44/100 | F4
45/100 | F5
46/100 | C#5
47/100 | F5
48/100 | D4
49/100 | F4
50/100 | D4.F3
51/100 | F3
52/100 | B-3
53/100 | B-3
54/100 | F3
55/100 | B-3
56/100 | F4
57/100 | G#3
58/100 | F5
59/100 | F5
60/100 | C#5
61/100 | C4
62/100 | C3
63/100 | B-3
64/100 | B-4
65/100 | F3
66/100 | F5
67/100 | B-3
68/100 | F3
69/100 | C#5
70/100 | B-3
71/100 | G#5
72/100 | F5
73/100 | F#3
74/100 | F#3
75/100 | C#5
76/100 | C#5
77/100 | B-5
78/100 | F#2
79/100 | G#2
80/100 

In [6]:
# Convert output to MIDI file
output_midi = music21.stream.Stream()
shift = 0
for pattern in prediction_output:
    if '.' in pattern:   # it's a chord
        notes_in_chord = pattern.split('.') # notes for the chord (numbers)
        chord_notes = [music21.note.Note() for _ in notes_in_chord] # real notes
        for i, note_in_chord in enumerate(notes_in_chord):
            chord_notes[i].pitch = music21.pitch.Pitch(note_in_chord)
        chord = music21.chord.Chord(chord_notes, quarterlength=0.5) # the chord per se
        output_midi.insert(shift, chord) # adding the chord to the stream
        shift += 0.3
    else:   # it's a note
        note = music21.note.Note(quarterlength=0.5)
        note.pitch = music21.pitch.Pitch(pattern)
        output_midi.insert(shift, note) # adding the note to the stream
        shift += 0.3

output_midi.write('midi', fp=OUTPUT_FILE)

'output.mid'